In [ ]:
%matplotlib tk
from Tkinter import TclError
import auto

In [ ]:
default_params = {'I': -12, 'gleak': 6.8, 'gKs': 50, 'gKf': 24.1, 'gNa': 100, 'gNaP': 0.8}

# Return a copy of the dict params, coplemented by default values for all parameters that are neither in params nor in exclude.
def add_default_params(params, exclude=None):
    if params is None: 
        params = {}
    else:
        params = params.copy()
        
    if exclude is None: exclude = []    
    for key, val in default_params.items():
        if key not in exclude:
            params.setdefault(key, val)
    return params

add_default_params({'gleak': 10}, exclude=['I'])

In [ ]:
# For all these functions, the AUTO constants can be overwritten at any time by using kwargs.

# Integrate the system using the Euler algorithm and return the last point.
def integrate_to_equilibrium(params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params))
    integration = auto.run(e='drosophila', c='integration', **kwargs)
    return integration()[-1]  # last point

# Continue equilibrium forwards and backwards in I from init point.
def continue_equilibrium(init, params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params, exclude=['I']))    
    equilibrium = auto.run(init, c='equilibrium', **kwargs)
    equilibrium += auto.run(init, c='equilibrium', DS='-', **kwargs)
    equilibrium = auto.relabel(auto.merge(equilibrium))  # get single curve with unique labels
    return equilibrium

# Continue limit cycle in I from init point (for example a Hopf bifurcation).
def continue_limit_cycle(init, params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params, exclude=['I']))
    return auto.run(init, c='tube', **kwargs)

# Continue bifurcation point in init for changing continuation_param. 
def continue_in_2_params(init, continuation_param, continuation_param_uzstop=None, params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params, exclude=['I', continuation_param]))
    kwargs.setdefault('ICP', ['I', continuation_param, 'PERIOD'])
    if continuation_param_uzstop is not None:
        kwargs.setdefault('UZSTOP', {'I': [-20, 100], continuation_param: continuation_param_uzstop})
    
    bifurcation_point = auto.run(init, c='2par', **kwargs)
    bifurcation_point += auto.run(init, c='2par', DS='-', **kwargs)
    bifurcation_point = auto.relabel(auto.merge(bifurcation_point))
    return bifurcation_point


# TODO: Make the plot functions simpler.

def plot_bifurcation_diagram(points, saveas=False, **kwargs):
    # AUTO raises a non-critical TclError (bad window path name) on Windows when showing the plot. Prevent its output by wrapping in a try block.
    try:
        auto.plot(points, bifurcation_y=['V', 'MAX V', 'MIN V'], stability=True, use_labels=False, 
                  grid=False, height=350, user_point_symbol='', xlabel='I / pA', ylabel='V / mV', color_list='blue blue green', **kwargs)
    except TclError:
        pass
    
    if saveas:
        p = auto.plot(points, bifurcation_y=['V', 'MAX V', 'MIN V'], stability=True, use_labels=False, 
                      grid=False, height=350, user_point_symbol='', xlabel='I / pA', ylabel='V / mV', color_list='blue blue green', 
                      hide=True, **kwargs)
        p.savefig(saveas, dpi=300)    
        
    
def plot_2_params_diagram(points, second_param, saveas=False, **kwargs):
    # AUTO raises a non-critical TclError (bad window path name) on Windows when showing the plot. Prevent its output by wrapping in a try block.
    try:
        auto.plot(points, bifurcation_x='I', bifurcation_y=second_param, use_labels=False, 
                  grid=False, height=350, xlabel='I / pA', **kwargs)
    except TclError:
        pass
    
    if saveas:
        p = auto.plot(points, bifurcation_x='I', bifurcation_y=second_param, use_labels=False, 
                      grid=False, height=350, xlabel='I / pA', hide=True, **kwargs)
        p.savefig(saveas, dpi=300)
    

Bifurcation diagram for default parameter values

In [ ]:
init = integrate_to_equilibrium()

In [ ]:
equilibrium = continue_equilibrium(init)

In [ ]:
#dir(equilibrium()[0])

In [ ]:
#equilibrium('LP2').data['PT']

In [ ]:
#dir(equilibrium)

In [ ]:
# Test for reading eigenvalues from fort.9
#equilibrium = continue_equilibrium(init)

In [ ]:
#print equilibrium.summary()
plot_bifurcation_diagram(equilibrium)

In [ ]:
tube = continue_limit_cycle(equilibrium('HB1'))

In [ ]:
# test case for reading multipliers
tube = continue_limit_cycle(equilibrium('HB1'))

In [ ]:
plot_bifurcation_diagram(equilibrium + tube)#, saveas='plots/bifurcation.png')

2 parameter continuation for gleak

In [ ]:
# Continue saddle-node bifurcation (LP1) for changing gleak (2-parameter continuation)
saddle_node_gleak = continue_in_2_params(equilibrium('LP1'), 'gleak', [0, 20])

In [ ]:
# Continue Hopf bifurcation (HB1) for changing gleak (2-parameter continuation)
hopf_gleak = continue_in_2_params(equilibrium('HB1'), 'gleak', UZSTOP={'I': [-20, 170], 'gleak': [0, 20]})

In [ ]:
plot_2_params_diagram(saddle_node_gleak + hopf_gleak, 'gleak', ylabel='gleak / nS', miny=0, saveas='plots/2par_gleak.png')

Bifurcation diagram for high gleak

In [ ]:
# tryout
new_gleak = 2
ini = integrate_to_equilibrium(params={'gleak': new_gleak, 'I': -100})
eq = continue_equilibrium(ini, params={'gleak': new_gleak}, UZSTOP={'I': [-150, 100]})
plot_bifurcation_diagram(eq)

In [ ]:
# As can be seen in the plot above, the system is governed by Hopf bifurcations for gleak >~ 8.6
# Make a bifurcation diagram for a high gleak value to see these bifurcations
# NOTE: I is shifted 50 pA in the plots
new_gleak = 11#8.84
params = {'gleak': new_gleak}
init_gleak = integrate_to_equilibrium(params={'gleak': new_gleak, 'I': 40})

In [ ]:
equilibrium_gleak = continue_equilibrium(init_gleak, params=params, UZSTOP={'I': [30, 150]})

In [ ]:
tube_gleak = continue_limit_cycle(equilibrium_gleak('HB1'), params=params, 
                             UZSTOP={'I': [30, 150], 'PERIOD': [0, 100]}, STOP=['LP2'], ILP=1)  # ILP=1 to detect folds

In [ ]:
plot_bifurcation_diagram(equilibrium_gleak + tube_gleak)#, saveas='plots/bifurcation_gleak={0}.png'.format(new_gleak))

2 parameter continuation for gKs

In [ ]:
saddle_node_gKs = continue_in_2_params(equilibrium('LP1'), 'gKs', [0, 300])

In [ ]:
hopf_gKs = continue_in_2_params(equilibrium('HB1'), 'gKs', UZSTOP={'I': [-20, 300], 'gKs': [0, 1300]})

In [ ]:
plot_2_params_diagram(saddle_node_gKs + hopf_gKs, 'gKs', ylabel='gKs / nS', miny=0, saveas='plots/2par_gKs.png')  #, minx=-20, maxx=120, miny=0, maxy=500)

2 parameter continuation for gKf

In [ ]:
saddle_node_gKf = continue_in_2_params(equilibrium('LP1'), 'gKf', [0, 2500])
saddle_node_gKf += continue_in_2_params(equilibrium('LP2'), 'gKf', [0, 2500])

In [ ]:
hopf_gKf = continue_in_2_params(equilibrium('HB1'), 'gKf', [0, 2500])

In [ ]:
plot_2_params_diagram(saddle_node_gKf + hopf_gKf, 'gKf', ylabel='gKf / nS', color_list='black black red', saveas='plots/2par_gKf.png')

2 parameter continuation for gNa

In [ ]:
saddle_node_gNa = continue_in_2_params(equilibrium('LP1'), 'gNa', [0, 1000])

In [ ]:
hopf_gNa = continue_in_2_params(equilibrium('HB1'), 'gNa', [0, 1000])

In [ ]:
plot_2_params_diagram(saddle_node_gNa + hopf_gNa, 'gNa', ylabel='gNa / nS', saveas='plots/2par_gNa.png')

2 parameter continuation for gNaP

In [ ]:
saddle_node_gNaP = continue_in_2_params(equilibrium('LP1'), 'gNaP', UZSTOP={'I': [-50, 100], 'gNaP': [-10, 10]})
saddle_node_gNaP += continue_in_2_params(equilibrium('LP2'), 'gNaP', UZSTOP={'I': [-50, 100], 'gNaP': [-10, 10]}) 

In [ ]:
hopf_gNaP = continue_in_2_params(equilibrium('HB1'), 'gNaP', UZSTOP={'I': [-50, 100], 'gNaP': [-10, 10]})

In [ ]:
plot_2_params_diagram(saddle_node_gNaP + hopf_gNaP, 'gNaP', ylabel='gNaP / nS', color_list='black black red', saveas='plots/2par_gNaP.png')

In [ ]:
params = {'gKs': 10}
init_gKs = integrate_to_equilibrium(params=params)

In [ ]:
equilibrium_gKs = continue_equilibrium(init_gKs, params=params, UZSTOP={'I': [-50, 100]})

In [ ]:
plot_bifurcation_diagram(equilibrium_gKs)

In [ ]:
tube_gKs = continue_limit_cycle(equilibrium_gKs('HB1'), params=params, STOP=['LP2'])  # ILP=1 to detect folds

In [ ]:
plot_bifurcation_diagram(equilibrium_gKs + tube_gKs)